# 250 : M2 Demo

In [ ]:
#!pip install ipywidgets

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
models = ['m2_joint_label_ref_cmbert_io',
 'm2_joint_label_ref_ptrn_cmbert_io',
 'm2_joint_label_ref_cmbert_iob2',
 'm2_joint_label_ref_ptrn_cmbert_iob2',
 'm2_joint_label_ocr_cmbert_io',
 'm2_joint_label_ocr_ptrn_cmbert_io',
 'm2_joint_label_ocr_cmbert_iob2',
 'm2_joint_label_ocr_ptrn_cmbert_iob2',
]

Choose an model on the HuggingFace among the following list :

In [ ]:
import ipywidgets as widgets

model_name = widgets.RadioButtons(
            options=models,
            layout={'width': 'max-content'}
        )
model_name

### Constants

In [ ]:
if 'iob2' in model_name.value:
    FORMAT = "IOB2"
else:
    FORMAT = 'IO'

In [ ]:
FINETUNED_MODEL = 'nlpso/' + model_name.value
FINETUNED_MODEL

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

LIMIT = 100

tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL) #Param : tokenizer du modele souhaité
model = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL) #Modèle choisi

#Classification des entités
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy=None, use_fast=True)

### Examples

In [ ]:
samples = """
☞  T Dufant (Victor), libraire, r. du Gros-Che- net. 2. JO \n
Dutay, essayeur du commerce, place Dau-  - píině, 5. DOB \n
Dulay, chandronnier, r. du Pont- aux- Chuux, 15. SI \n
Dufay (V.e), grenetière, r. du Fauh.-S. Denis, 20. 372 \n
Y ☞ Dnten,charentier, 1. Montmartre, 89. 318 \n
Dufey fils, bijoutier, passage de la Réunion A \n
Dnley, boucher, r. S.&quot;Louis-au-Marais,3. * \n
☞ Duffand, bijonteer, r. S.-Martin, 107. 284 \n
Duftaud, maçon, r. S-Honore  353. O \n
Dullaut, chandronnier, r. de la Sourdière I- ( ☞T4 \n
lukils. limonauier, 1. des Colonnes, 6. J9 \n
— T Dullocu, architecte, r. Basse-Porte-S.De- — Ris, 28. \n
Duflot, loueur de carrosses, r. de Paradis- 505 Poissonnière, 22. \n
Dufort, bottier, Palais-R., gal. vitrée, 215. A \n
Dufort fils, bottier (exp. 1819.), r. J.-J.. Rousscan. 18. 290 \n
* Dufort, tabletier, r. Jean-Pain-Mollet, 10. 4O \n
Dufossé, teiniurier, r. du Faub.-Montmar- tre , 63. 483 \n
Duſour el Besnard, march, de bois à bruler, quai de la Tournelle, 17. etr. des Fossés- SBernard. 11. Dí \n
Dufour jeune, march, de bois à brüler de la Pépinière, 53. 7 r. \n
Dufour (Mad.), march. de chapeaux de paitle, passage dn Panorama, 11. IO \n
Dutour, thapelier, r. S.-Honore 383. 3og \n
Dufour, cirier, r. de PArbre-Sec, 6o. 316 \n
Dufour et co., commiss., r. des Contures-  DAO S. Dervajs, 18. \n
Dulour, corropeir, 1. des Vertusy1d 3 \n
Dufour (Charles), épicier, r. S.-Denis, Jo7. ☞ 332 \n
Dufour, fab. d&apos;éventails (erp. 1819) Beaubourg, 48.  360 \n
Dufour, faiencter, r. S.-Anne , 50. 302 \n
Dufour, A. Sarnt,r. Ventadour, ☞7 \n
Dufour (Gabriel), libraire, r. de Vaugirard, A \n
Dufour, maçon, r. S.-Jean-Baptiste, 4 402 \n
Dufour, architecte, r. de Chartres-S.-Ho- 2781 40a nore, 12. (Elig.) \n
Dufour (Mad.), nouveautes, 1. Neuve-des- H12 Petits-Champs, 4. \n
Dufour Chabrol, papetier, r. S.-Martin ☞ 4 \n
Dufour (J.) et co,, fab. de papiers peints, ((1819); r. Beauvean-S. Antoine, 19. \n
Waring et Gillow Limited, ameublements an- glais, éctairage électrique et appareils sa- nitaires, r. Glück, 8 et boul. Haussmann, 31, TELÉPH] 244.45; succursale, r. Bleue, 7. WARISON�PARIS. TĚLÉPH 244.40. Adr.\n
Barruel (Paul), fabr. de produis chimiques et de siccatif zumatique, adjoint au maire du 15e arrond., 
Alleray, 70; dépót chez H. Suillot sle-Croix-de-la-Bretonnerič. 21 \n
Marchand (Mme) née Louise Scelle, en gros et détail, r. 77, et à Bercy, r. de Bourgogne, 25. d'Enfer, \n
Very (A.). ingenieur-constructeur, r. d'Angou- lême, 66, cité d'Angouléme, 6. TĚLÉPH 420.82. \n
Vacherie de Royaumont, Brémontler. 16; Chaussée-d'Antin, 68; Phals- bourg, 3 ; blv. Poissonnière, 1, et blv. St-Martin, 15. \n
Budan, inspect.-gén. des études, r. Charlot. 18 (au Marais). \n
"""

In [ ]:
from xmlize_util import get_NER_tags, xmlize_multilevel

for s in samples.split('\n'):
    if s != '':
        print(s)
        res = nlp(s)
        levels, num_levels = get_NER_tags(s,res,FORMAT)
        xml = xmlize_multilevel(levels,num_levels)
        print(xml)
        print('')